In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")


In [ ]:
df.head(5)


In [ ]:
df.shape

In [ ]:
df.isnull().sum()


In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


In [ ]:
sns.countplot('stroke',data=df)


In [ ]:
sns.countplot('stroke',data=df[df['ever_married']=='Yes'])

In [ ]:
sns.countplot('stroke',data=df[df['ever_married']=='No'])


In [ ]:
print('No. of strokes if patient is married =',df[df['ever_married']=='Yes']['stroke'].value_counts()[1]) 
print('No. of strokes if patient is not married =',df[df['ever_married']=='No']['stroke'].value_counts()[1])

In [ ]:
print('No. of strokes if patient is private employed =',df[df['work_type']=='Private']['stroke'].value_counts()[1]) 
print('No. of strokes if patient is self-employed =',df[df['work_type']=='Self-employed']['stroke'].value_counts()[1]) 
print('No. of strokes if patient is government employee =',df[df['work_type']=='Govt_job']['stroke'].value_counts()[1]) 
print('No. of strokes if patient is children =',df[df['work_type']=='children']['stroke'].value_counts()[1])
print('No. of strokes if patient is Never_worked =',df[df['work_type']=='Never_worked']['stroke'].value_counts()[0])

In [ ]:
print('No. of strokes if patient is Urban residence =',df[df['Residence_type']=='Urban']['stroke'].value_counts()[1]) 
print('No. of strokes if patient is Rural residence =',df[df['Residence_type']=='Rural']['stroke'].value_counts()[1])

In [ ]:
print('No. of strokes if patient had never smoked =',df[df['smoking_status']=='formerly smoked']['stroke'].value_counts()[1]) 
print('No. of strokes if patient had formely smoked =',df[df['smoking_status']=='never smoked']['stroke'].value_counts()[1]) 
print('No. of strokes if patient smokes =',df[df['smoking_status']=='smokes']['stroke'].value_counts()[1])

In [ ]:
import matplotlib.pyplot as plt
plt.bar(['male','female'], [df[df['gender']=='Male']['stroke'].value_counts()[1],df[df['gender']=='Female']['stroke'].value_counts()[1]])
plt.title('No. of strokes male vs female')

In [ ]:
plt.bar(['1','0'], [df[df['hypertension']==1]['stroke'].value_counts()[1],df[df['hypertension']==0]['stroke'].value_counts()[1]])
plt.title('have hypertension vs not having hypertension')

In [ ]:
plt.hist(df[df['stroke']==1]['age'])
plt.title('age vs stroke')

In [ ]:
df.work_type.unique()


In [ ]:
df.ever_married.unique()


In [ ]:
df.smoking_status.unique()


In [ ]:
plt.hist(df[df['stroke']==1]['avg_glucose_level'])
plt.title('avg_glucose_level vs stroke')

In [ ]:
df['bmi']=df['bmi'].replace(np.NaN,df['bmi'].mean())

In [ ]:
df=df.drop('id',axis=1)
df.head()

In [ ]:
df['gender'] = df['gender'].replace({'Male':0,'Female':1,'Other':2})
df['Residence_type'] = df['Residence_type'].replace({'Rural':0,'Urban':1})
df['work_type'] = df['work_type'].replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':3,'Never_worked':4})
df['smoking_status'] = df['smoking_status'].replace({'never smoked':0,'formerly smoked':1,'smokes':2,'Unknown':3})
df['ever_married'] = df['ever_married'].replace({'Yes':1,'No':0}).astype(np.uint8)


In [ ]:
df.head()

In [ ]:
a = pd.get_dummies(df['gender'], prefix = "gender")
b = pd.get_dummies(df['hypertension'], prefix = "hypertension")
c = pd.get_dummies(df['heart_disease'], prefix = "heart_disease")
d = pd.get_dummies(df['ever_married'],prefix='ever_married')
e = pd.get_dummies(df['work_type'],prefix='work_type')
f = pd.get_dummies(df['Residence_type'],prefix='Residence_type')
g = pd.get_dummies(df['smoking_status'],prefix='smoking_status')
df = df.drop(columns = ['gender', 'hypertension', 'heart_disease','ever_married','work_type','Residence_type','smoking_status'])
# data.head()

In [ ]:
df2 = [df, a, b, c,d,e,f,g]
df = pd.concat(df2, axis = 1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
x=df.drop('stroke',axis=1)
y=df['stroke']

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x, y= oversample.fit_resample(x, y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
pd.set_option('display.max_columns', None)

# df = df.drop(columns = ['gender', 'hypertension', 'heart_disease','ever_married','work_type','Residence_type','smoking_status'])
# df.head()

In [ ]:
x_train.head()

In [ ]:
x_train.shape

In [ ]:
y_train.unique()

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import MinMaxScaler

std_scaler = MinMaxScaler()
# df4=df.drop('stroke',axis=1)
x_train= pd.DataFrame(std_scaler.fit_transform(x_train), columns=x_train.columns)
# df= pd.DataFrame(std_scaler.fit_transform(df), columns=df.columns)

In [ ]:
x_test= pd.DataFrame(std_scaler.transform(x_test), columns=x_test.columns)

In [ ]:
x_test.head()

In [ ]:
from pickle import dump
dump(std_scaler, open('stroke_scaler.pkl', 'wb'))

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
x_train.shape

In [ ]:
x_train.tail()

In [ ]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz 
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier()
}

In [ ]:
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(x_train, y_train)
    training_score = cross_val_score(classifier, x_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

In [ ]:
from sklearn.model_selection import GridSearchCV


# Logistic Regression 
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}



grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params,cv=5)
grid_log_reg.fit(x_train, y_train)
#We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_

knears_params = {"n_neighbors": list(range(2,7,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params,cv=5)
grid_knears.fit(x_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_

# # Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params,cv=5)
grid_svc.fit(x_train, y_train)

# # SVC best estimator
svc = grid_svc.best_estimator_

# # DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,6,1)), 
              "min_samples_leaf": list(range(5,7,1))}
grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params,cv=5)
grid_tree.fit(x_train, y_train)

# # tree best estimator
tree_clf = grid_tree.best_estimator_

##random forest classifier
rfc=RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid,cv=5)
CV_rfc.fit(x_train, y_train)
random_tree_clf=CV_rfc.best_estimator_

In [ ]:
print("logestic regression best parameter",grid_log_reg.best_params_)
print("logestic regression best parameter",grid_knears.best_params_)
print("logestic regression best parameter",grid_svc.best_params_)
print("logestic regression best parameter",grid_tree.best_params_)
print("logestic regression best parameter",CV_rfc.best_params_)

In [ ]:
log_reg_score = cross_val_score(log_reg, x_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')


knears_score = cross_val_score(knears_neighbors, x_train, y_train, cv=5)
print('Knears Neighbors Cross Validation Score', round(knears_score.mean() * 100, 2).astype(str) + '%')
svc_score = cross_val_score(svc, x_train, y_train, cv=5)
print('Support Vector Classifier Cross Validation Score', round(svc_score.mean() * 100, 2).astype(str) + '%')

tree_score = cross_val_score(tree_clf, x_train, y_train, cv=5)
print('DecisionTree Classifier Cross Validation Score', round(tree_score.mean() * 100, 2).astype(str) + '%')
random_forest_score = cross_val_score(random_tree_clf, x_train, y_train, cv=5)
print('Random forest Classifier Cross Validation Score', round(random_forest_score.mean() * 100, 2).astype(str) + '%')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict

log_reg_pred = cross_val_predict(log_reg, x_train, y_train, cv=5,
                             method="decision_function")

knears_pred = cross_val_predict(knears_neighbors, x_train, y_train, cv=5)

svc_pred = cross_val_predict(svc, x_train, y_train, cv=5,
                             method="decision_function")

tree_pred = cross_val_predict(tree_clf, x_train, y_train, cv=5)
random_forest_pred = cross_val_predict(random_tree_clf, x_train, y_train, cv=5)

In [ ]:
from sklearn.metrics import roc_auc_score


In [ ]:
log_fpr, log_tpr, log_thresold = roc_curve(y_train, log_reg_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_train, knears_pred)
svc_fpr, svc_tpr, svc_threshold = roc_curve(y_train, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_train, tree_pred)
random_forest_fpr, random_forest_tpr, random_forest_threshold = roc_curve(y_train, random_forest_pred)
def graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr,random_forest_fpr,random_forest_tpr):
    plt.figure(figsize=(16,8))
    plt.title('ROC Curve \n Top 4 Classifiers', fontsize=18)
    plt.plot(log_fpr, log_tpr, label='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_train, log_reg_pred)))
    plt.plot(knear_fpr, knear_tpr, label='KNears Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_train, knears_pred)))
    plt.plot(svc_fpr, svc_tpr, label='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_train, svc_pred)))
    plt.plot(tree_fpr, tree_tpr, label='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_train, tree_pred)))
    plt.plot(random_forest_fpr, random_forest_tpr, label='Random forest Classifier Score: {:.4f}'.format(roc_auc_score(y_train, random_forest_pred)))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.01, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()
    
graph_roc_curve_multiple(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr,random_forest_fpr,random_forest_tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

print('Logistic Regression: ', roc_auc_score(y_train, log_reg_pred))
print('KNears Neighbors: ', roc_auc_score(y_train, knears_pred))
print('Support Vector Classifier: ', roc_auc_score(y_train, svc_pred))
print('Decision Tree Classifier: ', roc_auc_score(y_train, tree_pred))
print('Random Forest Classifier: ', roc_auc_score(y_train, random_forest_pred))

In [ ]:
x_test

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
y_pred_log_reg = log_reg.predict(x_test)

y_pred_knear = knears_neighbors.predict(x_test)
y_pred_svc = svc.predict(x_test)
y_pred_tree = tree_clf.predict(x_test)
y_pred_forest = random_tree_clf.predict(x_test)

log_reg_cf = confusion_matrix(y_test, y_pred_log_reg)
kneighbors_cf = confusion_matrix(y_test, y_pred_knear)
svc_cf = confusion_matrix(y_test, y_pred_svc)
tree_cf = confusion_matrix(y_test, y_pred_tree)
forest_cf = confusion_matrix(y_test, y_pred_forest)


fig, ax = plt.subplots(2, 2,figsize=(22,12))


sns.heatmap(log_reg_cf, ax=ax[0][0], annot=True, cmap=plt.cm.copper)
ax[0, 0].set_title("Logistic Regression \n Confusion Matrix", fontsize=14)
ax[0, 0].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[0, 0].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(kneighbors_cf, ax=ax[0][1], annot=True, cmap=plt.cm.copper)
ax[0][1].set_title("KNearsNeighbors \n Confusion Matrix", fontsize=14)
ax[0][1].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[0][1].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(svc_cf, ax=ax[1][0], annot=True, cmap=plt.cm.copper)
ax[1][0].set_title("Suppor Vector Classifier \n Confusion Matrix", fontsize=14)
ax[1][0].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[1][0].set_yticklabels(['', ''], fontsize=14, rotation=360)

sns.heatmap(tree_cf, ax=ax[1][1], annot=True, cmap=plt.cm.copper)
ax[1][1].set_title("DecisionTree Classifier \n Confusion Matrix", fontsize=14)
ax[1][1].set_xticklabels(['', ''], fontsize=14, rotation=90)
ax[1][1].set_yticklabels(['', ''], fontsize=14, rotation=360)


plt.show()

In [ ]:
sns.heatmap(forest_cf,annot=True, cmap=plt.cm.copper)


In [ ]:
from sklearn.metrics import classification_report,f1_score


print('Logistic Regression:')
print(classification_report(y_test, y_pred_log_reg))

print('KNears Neighbors:')
print(classification_report(y_test, y_pred_knear))

print('Support Vector Classifier:')
print(classification_report(y_test, y_pred_svc))

print('Decision Tree Classifier:')
print(classification_report(y_test, y_pred_tree))
print('random forest Support Vector Classifier:')
print(classification_report(y_test, y_pred_forest))

In [ ]:
from collections import Counter
Counter(y_test)

In [ ]:
 f1_score(y_test,y_pred_log_reg )

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict

log_reg_pred_test = cross_val_predict(log_reg, x_test, y_test, cv=5,
                             method="decision_function")

knears_pred_test = cross_val_predict(knears_neighbors, x_test, y_test, cv=5)

svc_pred_test = cross_val_predict(svc, x_test, y_test, cv=5,
                             method="decision_function")

tree_pred_test = cross_val_predict(tree_clf, x_test, y_test, cv=5)
random_forest_pred_test = cross_val_predict(random_tree_clf, x_test, y_test, cv=5)

In [ ]:

print('Logistic Regression: ', roc_auc_score(y_test, log_reg_pred_test))
print('KNears Neighbors: ', roc_auc_score(y_test, knears_pred_test))
print('Support Vector Classifier: ', roc_auc_score(y_test, svc_pred_test))
print('Decision Tree Classifier: ', roc_auc_score(y_test, tree_pred_test))
print('Random Forest Classifier: ', roc_auc_score(y_test, random_forest_pred_test))

In [ ]:
log_fpr_test, log_tpr_test, log_thresold_test = roc_curve(y_test, log_reg_pred_test)
knear_fpr_test, knear_tpr_test, knear_threshold_test = roc_curve(y_test, knears_pred_test)
svc_fpr_test, svc_tpr_test, svc_threshold_test = roc_curve(y_test, svc_pred_test)
tree_fpr_test, tree_tpr_test, tree_threshold_test= roc_curve(y_test, tree_pred_test)
random_forest_fpr_test, random_forest_tpr_test, random_forest_threshold_test = roc_curve(y_test, random_forest_pred_test)

In [ ]:
def graph_roc_curve_multiple_test(log_fpr, log_tpr, knear_fpr, knear_tpr, svc_fpr, svc_tpr, tree_fpr, tree_tpr,random_forest_fpr,random_forest_tpr):
    plt.figure(figsize=(16,8))
    plt.title('ROC Curve \n Top 4 Classifiers', fontsize=18)
    plt.plot(log_fpr, log_tpr, label='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_test, log_reg_pred_test)))
    plt.plot(knear_fpr, knear_tpr, label='KNears Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_test, knears_pred_test)))
    plt.plot(svc_fpr, svc_tpr, label='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_test, svc_pred_test)))
    plt.plot(tree_fpr, tree_tpr, label='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_test, tree_pred_test)))
    plt.plot(random_forest_fpr, random_forest_tpr, label='Random forest Classifier Score: {:.4f}'.format(roc_auc_score(y_test, random_forest_pred_test)))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.01, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()

In [ ]:
graph_roc_curve_multiple_test(log_fpr_test, log_tpr_test, knear_fpr_test, knear_tpr_test, svc_fpr_test, svc_tpr_test, tree_fpr_test, tree_tpr_test,random_forest_fpr_test,random_forest_tpr_test)
plt.show()

In [ ]:
# import xgboost as xgb
# model = xgb.XGBClassifier()
# model.fit(x_train,y_train)

In [ ]:
# y_pred1 = model.predict(_test)

In [ ]:
# roc_auc_score(y_test, y_pred1)

In [ ]:
# cm = confusion_matrix(y_test, y_pred1)
# cm

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold
# from sklearn.impute import KNNImputer
# from sklearn.tree import *
# from sklearn.metrics import f1_score

In [ ]:

# imputer = KNNImputer(n_neighbors=2)
# X_train = imputer.fit_transform(x_train)
# X_test = imputer.fit_transform(x_test)
    
# clf = DecisionTreeClassifier()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# f = f1_score(y_true = y_test , y_pred = y_pred,average = 'weighted')
    
# print(f)

In [ ]:
# print('Logistic Regression:')
# print(classification_report(y_test, y_pred))

In [ ]:
# cm = confusion_matrix(y_test, y_pred)
# cm

In [ ]:
dump(random_tree_clf, open('stroke_model.pkl', 'wb'))


In [ ]:
# 62861
# Female
# 78
# 0
# 0
# Yes
# Private
# Urban
# 67.29
# 24.6
# never smoked
# 1

In [ ]:
from pickle import load
stroke_scaler = load(open('stroke_scaler.pkl', 'rb'))
model=load(open("./stroke_model.pkl",'rb'))

In [ ]:
data = [{'gender': 'Female', 'age': 78, 'hypertension':1,'heart_disease':1, 'ever_married':'Yes', 'work_type':'Private', 'Residence_type':'Urban', 'avg_glucose_level':10.29, 'bmi':24.2, 'smoking_status':'never smoked'}]

In [ ]:
df3=pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")


In [ ]:
test = pd.DataFrame(data)
test.head()

In [ ]:
train_objs_num = len(df3)
df3=df3.drop(['id','stroke'],axis=1)
data = pd.concat(objs=[df3, test], axis=0)
data['gender'] = data['gender'].replace({'Male':0,'Female':1,'Other':2})
data['Residence_type'] = data['Residence_type'].replace({'Rural':0,'Urban':1})
data['work_type'] = data['work_type'].replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':3,'Never_worked':4})
data['smoking_status'] = data['smoking_status'].replace({'never smoked':0,'formerly smoked':1,'smokes':2,'Unknown':3})
data['ever_married'] = data['ever_married'].replace({'Yes':1,'No':0}).astype(np.uint8)
a = pd.get_dummies(data['gender'], prefix = "gender")
b = pd.get_dummies(data['hypertension'], prefix = "hypertension")
c = pd.get_dummies(data['heart_disease'], prefix = "heart_disease")
d = pd.get_dummies(data['ever_married'],prefix='ever_married')
e = pd.get_dummies(data['work_type'],prefix='work_type')
f = pd.get_dummies(data['Residence_type'],prefix='Residence_type')
g = pd.get_dummies(data['smoking_status'],prefix='smoking_status')
temp = [data, a, b, c,d,e,f,g]

data = pd.concat(temp, axis = 1)
data = data.drop(columns = ['gender', 'hypertension', 'heart_disease','ever_married','work_type','Residence_type','smoking_status'])
# data= pd.DataFrame(std_scaler.fit_transform(data), columns=data.columns)

In [ ]:
rest = data[:train_objs_num]
test = data[train_objs_num:]

In [ ]:
test

In [ ]:
test_scaled = stroke_scaler.transform(test)


In [ ]:
test_scaled

In [ ]:
test_prediction = model.predict(test_scaled)[0]


In [ ]:
test_prediction

In [ ]:
rest['bmi']=rest['bmi'].replace(np.NaN,rest['bmi'].mean())

In [ ]:
rest_scaled= stroke_scaler.transform(rest)

In [ ]:
rest_prediction = model.predict(rest_scaled)


In [ ]:
cm = confusion_matrix(df['stroke'], rest_prediction)
cm